In [1]:
# Import all the libraries I need
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

# ignore Deprecation Warning
import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning) 

from sklearn.ensemble import RandomForestRegressor
#from sklearn.ensemble import RandomForestClassifier
#from xgboost import XGBClassifier
#from sklearn.model_selection import cross_val_score
#from sklearn.model_selection import GridSearchCV

import keras 
from keras.models import Sequential # intitialize the ANN
from keras.layers import Dense      # create layers


Using TensorFlow backend.
/home/pi/anaconda3/envs/nitp-hack/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:526: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/pi/anaconda3/envs/nitp-hack/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:527: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/pi/anaconda3/envs/nitp-hack/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:528: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/home/pi/anaconda3/envs

In [88]:

# load the data
df_train = pd.read_csv('train.csv')
df_test = pd.read_csv('test.csv')
df = df_train.append(df_test , ignore_index = True)

# some quick inspections
df_train.shape, df_test.shape, df_train.columns.values

((891, 12),
 (418, 11),
 array(['PassengerId', 'Survived', 'Pclass', 'Name', 'Sex', 'Age', 'SibSp',
        'Parch', 'Ticket', 'Fare', 'Cabin', 'Embarked'], dtype=object))

## Cleaning

In [77]:

df['Title'] = df['Name'].map( lambda x: x.split(',')[1].split( '.' )[0].strip())

df['Title'] = df['Title'].replace('Mlle', 'Miss')
df['Title'] = df['Title'].replace(['Mme','Lady','Ms'], 'Mrs')
df.Title.loc[ (df.Title !=  'Master') & (df.Title !=  'Mr') & (df.Title !=  'Miss') 
             & (df.Title !=  'Mrs')] = 'Others'


df = pd.concat([df, pd.get_dummies(df['Title'])], axis=1).drop(labels=['Name'], axis=1)

# map the two genders to 0 and 1
df.Sex = df.Sex.map({'male':0, 'female':1})

# create a new feature "Family"
df['Family'] = df['SibSp'] + df['Parch'] + 1

df.Family = df.Family.map(lambda x: 0 if x > 4 else x)

df.Ticket = df.Ticket.map(lambda x: x[0])

guess_Fare = df.Fare.loc[ (df.Ticket == '3') & (df.Pclass == 3) & (df.Embarked == 'S')].median()
df.Fare.fillna(guess_Fare , inplace=True)

# inspect the mean Fare values for people who died and survived
df[['Fare', 'Survived']].groupby(['Survived'],as_index=False).mean()

# bin Fare into five intervals with equal amount of people
df['Fare-bin'] = pd.qcut(df.Fare,5,labels=[1,2,3,4,5]).astype(int)


# notice that instead of using Title, we should use its corresponding dummy variables 
df_sub = df[['Age','Master','Miss','Mr','Mrs','Others','Fare-bin','SibSp']]

X_train  = df_sub.dropna().drop('Age', axis=1)
y_train  = df['Age'].dropna()
X_test = df_sub.loc[np.isnan(df.Age)].drop('Age', axis=1)

regressor = RandomForestRegressor(n_estimators = 300)
regressor.fit(X_train, y_train)
y_pred = np.round(regressor.predict(X_test),1)
df.Age.loc[df.Age.isnull()] = y_pred

bins = [ 0, 4, 12, 18, 30, 50, 65, 100] # This is somewhat arbitrary...
age_index = (1,2,3,4,5,6,7)
#('baby','child','teenager','young','mid-age','over-50','senior')
df['Age-bin'] = pd.cut(df.Age, bins, labels=age_index).astype(int)

df['Ticket'] = df['Ticket'].replace(['A','W','F','L','5','6','7','8','9'], '4')

df = df.drop(labels=['Cabin'], axis=1)

# fill the NAN
df.Embarked.fillna('S' , inplace=True )

df = df.drop(labels='Embarked', axis=1)

# dummy encoding
df = pd.get_dummies(df,columns=['Ticket'])

df = df.drop(labels=['SibSp','Parch','Age','Fare','Title','PassengerId'], axis=1)

/home/pi/anaconda3/envs/nitp-hack/lib/python3.7/site-packages/pandas/core/indexing.py:670: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_with_indexer(indexer, value)
/home/pi/anaconda3/envs/nitp-hack/lib/python3.7/site-packages/pandas/core/indexing.py:670: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_with_indexer(indexer, value)


In [89]:
def preprocessing(dfInput):
    df = dfInput.copy()
    df['Title'] = df.Name.map(lambda x: x.split(',')[1].split('.')[0].strip())

    df['Title'] = df['Title'].replace('Mlle', 'Miss')
    df['Title'] = df['Title'].replace(['Mme', 'Lady', 'Ms'], 'Mrs')
    df.Title.loc[(df.Title != 'Master') & (df.Title != 'Mr') & (df.Title != 'Miss')
                 & (df.Title != 'Mrs')] = 'Others'
    df = pd.concat([df, pd.get_dummies(df['Title'])],
                   axis=1).drop(labels=['Name'], axis=1)

    # map the two genders to 0 and 1
    df.Sex = df.Sex.map({'male': 0, 'female': 1})

    # create a new feature "Family"
    df['Family'] = df['SibSp'] + df['Parch'] + 1

    df.Family = df.Family.map(lambda x: 0 if x > 4 else x)

    df.Ticket = df.Ticket.map(lambda x: x[0])

    guess_Fare = df.Fare.loc[(df.Ticket == '3') & (
        df.Pclass == 3) & (df.Embarked == 'S')].median()
    df.Fare.fillna(guess_Fare, inplace=True)

    # inspect the mean Fare values for people who died and survived
    df[['Fare', 'Survived']].groupby(['Survived'], as_index=False).mean()

    # bin Fare into five intervals with equal amount of people
    df['Fare-bin'] = pd.qcut(df.Fare, 5, labels=[1, 2, 3, 4, 5]).astype(int)

    # notice that instead of using Title, we should use its corresponding dummy variables
    df_sub = df[['Age', 'Master', 'Miss', 'Mr',
                 'Mrs', 'Others', 'Fare-bin', 'SibSp']]

    X_train = df_sub.dropna().drop('Age', axis=1)
    y_train = df['Age'].dropna()
    X_test = df_sub.loc[np.isnan(df.Age)].drop('Age', axis=1)

    regressor = RandomForestRegressor(n_estimators=300)
    regressor.fit(X_train, y_train)
    y_pred = np.round(regressor.predict(X_test), 1)
    df.Age.loc[df.Age.isnull()] = y_pred

    bins = [0, 4, 12, 18, 30, 50, 65, 100]  # This is somewhat arbitrary...
    age_index = (1, 2, 3, 4, 5, 6, 7)
    # ('baby','child','teenager','young','mid-age','over-50','senior')
    df['Age-bin'] = pd.cut(df.Age, bins, labels=age_index).astype(int)

    df['Ticket'] = df['Ticket'].replace(
        ['A', 'W', 'F', 'L', '5', '6', '7', '8', '9'], '4')

    df = df.drop(labels=['Cabin'], axis=1)

    # fill the NAN
    df.Embarked.fillna('S', inplace=True)

    df = df.drop(labels='Embarked', axis=1)

    # dummy encoding
    df = pd.get_dummies(df, columns=['Ticket'])

    df = df.drop(labels=['SibSp','Parch','Age','Fare','Title','PassengerId'], axis=1)
    return df.copy()

In [90]:
df = preprocessing(df)

/home/pi/anaconda3/envs/nitp-hack/lib/python3.7/site-packages/pandas/core/indexing.py:670: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_with_indexer(indexer, value)
/home/pi/anaconda3/envs/nitp-hack/lib/python3.7/site-packages/pandas/core/indexing.py:670: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_with_indexer(indexer, value)


In [83]:
list(enumerate(pd.read_csv('train.csv').columns))

[(0, 'PassengerId'),
 (1, 'Survived'),
 (2, 'Pclass'),
 (3, 'Name'),
 (4, 'Sex'),
 (5, 'Age'),
 (6, 'SibSp'),
 (7, 'Parch'),
 (8, 'Ticket'),
 (9, 'Fare'),
 (10, 'Cabin'),
 (11, 'Embarked')]

In [91]:

list(enumerate(df.columns))

[(0, 'Survived'),
 (1, 'Pclass'),
 (2, 'Sex'),
 (3, 'Master'),
 (4, 'Miss'),
 (5, 'Mr'),
 (6, 'Mrs'),
 (7, 'Others'),
 (8, 'Family'),
 (9, 'Fare-bin'),
 (10, 'Age-bin'),
 (11, 'Ticket_1'),
 (12, 'Ticket_2'),
 (13, 'Ticket_3'),
 (14, 'Ticket_4'),
 (15, 'Ticket_C'),
 (16, 'Ticket_P'),
 (17, 'Ticket_S')]

In [84]:
df.to_csv('fullData_clean.csv', )

[(0, 'Survived'),
 (1, 'Pclass'),
 (2, 'Sex'),
 (3, 'Master'),
 (4, 'Miss'),
 (5, 'Mr'),
 (6, 'Mrs'),
 (7, 'Others'),
 (8, 'Family'),
 (9, 'Fare-bin'),
 (10, 'Age-bin'),
 (11, 'Ticket_1'),
 (12, 'Ticket_2'),
 (13, 'Ticket_3'),
 (14, 'Ticket_4'),
 (15, 'Ticket_C'),
 (16, 'Ticket_P'),
 (17, 'Ticket_S')]

In [69]:
df1 = pd.read_csv('train.csv')
df1

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S
...,...,...,...,...,...,...,...,...,...,...,...,...
886,887,0,2,"Montvila, Rev. Juozas",male,27.0,0,0,211536,13.0000,NaN,S
887,888,1,1,"Graham, Miss. Margaret Edith",female,19.0,0,0,112053,30.0000,B42,S
888,889,0,3,"Johnston, Miss. Catherine Helen ""Carrie""",female,NaN,1,2,W./C. 6607,23.4500,NaN,S
889,890,1,1,"Behr, Mr. Karl Howell",male,26.0,0,0,111369,30.0000,C148,C


## Modeling and Prediction
Now we can drop the features we don't need and split the data into training and test sets

In [ ]:

y_train = df[0:891]['Survived'].values
X_train = df[0:891].drop(['Survived','PassengerId'], axis=1).values
X_test  = df[891:].drop(['Survived','PassengerId'], axis=1).values

(09/12/2017 update) Using NN gives better result than XGBoost and Random Forest do. 

In [ ]:
# Initialising the NN
def getModel():
    model = Sequential()

    # layers
    model.add(Dense(units = 9, kernel_initializer = 'uniform', activation = 'relu', input_dim = 17))
    model.add(Dense(units = 9, kernel_initializer = 'uniform', activation = 'relu'))
    model.add(Dense(units = 5, kernel_initializer = 'uniform', activation = 'relu'))
    model.add(Dense(units = 1, kernel_initializer = 'uniform', activation = 'sigmoid'))

    # Compiling the ANN
    model.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])
    return model

# Train the ANN
model.fit(X_train, y_train, batch_size = 32, epochs = 200)

We can now get the prediction. I got a public score of 0.81339 using the output from my laptop (python 2.7), which is different from what is generated here.

In [ ]:
y_pred = model.predict(X_test)
y_final = (y_pred > 0.5).astype(int).reshape(X_test.shape[0])

output = pd.DataFrame({'PassengerId': df_test['PassengerId'], 'Survived': y_final})
output.to_csv('prediction-ann.csv', index=False)